# Trading Strategy Backtests

A backtest of a trading strategy using the VectorBT library on tick data from January 2023 to May 2023.

In [1]:
# Import libraries
import vectorbt as vbt
import pandas as pd
import numpy as np
import datetime
import talib
from numba import njit

In [2]:
# Load and observe data. Make a timestamp as index
df = pd.read_csv('crypto_2023_upto_May8.csv')
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

df.head()

,open,high,low,close,volume,dir,dist,x_open,x_high,x_low,x_close,x_dir,x_dist
time,,,,,,,,,,,,,
2023-01-01 00:00:00,0.7578,0.7579,0.7573,0.7575,29643,-1,0.0006,-0.000010,-0.000010,-0.000397,-0.000194,-1,0.000386
2023-01-01 00:01:00,0.7578,0.7579,0.7574,0.7575,2377,-1,0.0005,0.000083,0.000083,-0.000456,-0.000298,-1,0.000539
2023-01-01 00:02:00,0.7573,0.7574,0.7568,0.7568,4221,-1,0.0006,-0.000348,-0.000189,-0.000485,-0.000485,-1,0.000296
2023-01-01 00:03:00,0.7568,0.7569,0.7566,0.7566,4470,-1,0.0003,-0.000535,-0.000447,-0.000592,-0.000592,-1,0.000145
2023-01-01 00:04:00,0.7565,0.7565,0.7556,0.7562,15390,-1,0.0009,-0.000537,0.000000,-0.000537,0.000000,1,0.000537


In [3]:
# Check for NaN values
print(df.isna().value_counts())

open   high   low    close  volume  dir    dist   x_open  x_high  x_low  x_close  x_dir  x_dist
False  False  False  False  False   False  False  False   False   False  False    False  False     182954
dtype: int64


In [4]:
# Check data values
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 182954 entries, 2023-01-01 00:00:00 to 2023-05-08 23:59:00
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   open     182954 non-null  float64
 1   high     182954 non-null  float64
 2   low      182954 non-null  float64
 3   close    182954 non-null  float64
 4   volume   182954 non-null  int64  
 5   dir      182954 non-null  int64  
 6   dist     182954 non-null  float64
 7   x_open   182954 non-null  float64
 8   x_high   182954 non-null  float64
 9   x_low    182954 non-null  float64
 10  x_close  182954 non-null  float64
 11  x_dir    182954 non-null  int64  
 12  x_dist   182954 non-null  float64
dtypes: float64(10), int64(3)
memory usage: 19.5 MB


In [5]:
# shift close price by 10min compare it and place its value into classification equasion to get 'trend'
# 1 = long, -1 short, 0 equal.
df['ten_dir'] = np.where(df['close'] > df['close'].shift(10), 1,
                 np.where(df['close'] < df['close'].shift(10), -1, 0))


In [6]:
# Remove first 10 rows that have no data
df = df[10:]

#### Extract the necessary value series from the dataframe.

In [7]:
# Pull close price values
price = df.close
price.head()

time
2023-01-01 00:10:00    0.7547
2023-01-01 00:11:00    0.7545
2023-01-01 00:12:00    0.7542
2023-01-01 00:13:00    0.7541
2023-01-01 00:14:00    0.7537
Name: close, dtype: float64

In [8]:
# Pull x_dist values
x_dist = df.x_dist
x_dist.head()

time
2023-01-01 00:10:00    0.000535
2023-01-01 00:11:00    0.000255
2023-01-01 00:12:00    0.000287
2023-01-01 00:13:00    0.000293
2023-01-01 00:14:00    0.000375
Name: x_dist, dtype: float64

In [9]:
# Volume values
vol = df.volume
vol.head()

time
2023-01-01 00:10:00    11628
2023-01-01 00:11:00     6188
2023-01-01 00:12:00    23797
2023-01-01 00:13:00     2935
2023-01-01 00:14:00     7356
Name: volume, dtype: int64

In [10]:
# Trend past 10 min
ten = df.ten_dir
ten.head()

time
2023-01-01 00:10:00   -1
2023-01-01 00:11:00   -1
2023-01-01 00:12:00   -1
2023-01-01 00:13:00   -1
2023-01-01 00:14:00   -1
Name: ten_dir, dtype: int32

### Create trading signal function


In [11]:
# Make a function that finds outlier for n period with p percentile
@njit
def find_outlier(x_dist, n, p):
    # Create an array in a shape of x_dist with all False values
    outlier_flags = np.zeros_like(x_dist)
    
    for i in range(n, len(x_dist)):
        start_index = i - n
        end_index = i
        period = x_dist[start_index:end_index]
        
        # Calculate the threshold based on the given percentile
        threshold = np.percentile(period, p)
        
        if x_dist[i] >= threshold:
            outlier_flags[i] = True
            
    return outlier_flags 

In [12]:
# Function check
x = np.array([1,1,2,1,8,1,2,1,2,1,1,2,1,1,12])
n = 4
p = 75

outlier_flags = find_outlier(x, n, p)

print(outlier_flags)

[0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]


### VectorBT setup for the multiparameter backtesting.
Trading logic function, custom indicators, parameters ranges and output charts.

In [ ]:
# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(200,700, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=70000, 
        window = windows,
        n = ns,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
print(returns.max())
print(returns.idxmax())


In [ ]:
returns = returns.groupby(level=['x_n', 'x_window']).mean()
# Heatmap plot of input parameters
fig = pf.total_return().vbt.heatmap(
        x_level = 'x_window', # name of columns in printout of portfolio
        y_level = 'x_n',
        ) # slider option for heatmap to swith between symbols

fig.show()

In [ ]:
# Volume values and n backtests

# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(200,700, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=vols, 
        window = 15,
        n = ns,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
print(returns.max())
print(returns.idxmax())

In [ ]:
returns = returns.groupby(level=['x_n', 'x_vol_val']).mean()
# Heatmap plot of input parameters
fig = pf.total_return().vbt.heatmap(
        x_level = 'x_vol_val', # name of columns in printout of portfolio
        y_level = 'x_n',
        ) # slider option for heatmap to swith between symbols

fig.show()

In [ ]:
# Closer look at n values [640,800]. Vol 120, window 15

# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(640,800, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=120000, 
        window = 15,
        n = ns,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
print(returns.max())
print(returns.idxmax())

In [ ]:
returns = returns.groupby(level=['x_n', 'x_vol_val']).mean()
# Heatmap plot of input parameters
fig = pf.total_return().vbt.heatmap(
        x_level = 'x_vol_val', # name of columns in printout of portfolio
        y_level = 'x_n',
        ) # slider option for heatmap to swith between symbols

fig.show()

#### Best found parameters

In [13]:
# Define RSI function using vbt wrap function and extracting RSI from talib
RSI = vbt.IndicatorFactory.from_talib('RSI')

# Define indicator and strategy
def optimize_x(price,x_dist, ten, vol,vol_val,window, n, p):
   
    # function
    outlier = find_outlier(x_dist,n,p)
    # Short Entries
    short = ((outlier == True) & (ten == 1) & (vol > vol_val))
    # Long entries
    long = ((outlier == True) & (ten == -1) & (vol > vol_val))
    
    #grab values of rsi
    rsi = RSI.run(price, window).real # .real means getting outcome values
    #Exits
    short_exit = (rsi <= 30)
    long_exit = (rsi >= 70)
    
    return long, long_exit, short, short_exit

# Create indicator function that uses defined strategy
x_ind = vbt.IndicatorFactory(
        class_name = 'optimizeX',
        short_name = 'x',
        input_names = ['price','x_dist', 'ten', 'vol'],
        param_names = ['vol_val','window', 'n', 'p'],
        output_names = ['entries', 'long_exit','short_entries','short_exit' ]
        ).from_apply_func(
                optimize_x,
                vol_val = 70000,
                window = 14,
                n = 120,
                p = 100,
                keep_pd=False)

# Create ranges of our tested parameters
ns = np.arange(640,800, step=20, dtype=int)
ps = np.arange(80,100, step=2, dtype=int)
windows = np.arange(12,20, step=1, dtype=int)
vols = np.arange(60000,200000, step=20000, dtype=int)

# Define results
res = x_ind.run(
        price,
        x_dist,
        ten,
        vol,
        vol_val=120000, 
        window = 15,
        n = 700,
        p = 100,
        param_product=True
        ) 

long_entries = res.entries
short_entries = res.short_entries
long_exits = res.long_exit
short_exits = res.short_exit


# Build portfolio
pf = vbt.Portfolio.from_signals(
    price,
    long_entries,
    long_exits,
    short_entries,
    short_exits,
    sl_stop=0.08,
    
    upon_dir_conflict = vbt.portfolio.enums.DirectionConflictMode.Ignore,
    upon_opposite_entry = vbt.portfolio.enums.OppositeEntryMode.Reverse,
    freq='1T', fees = 0.0003)


returns = pf.total_return()
#print(returns.max())
#pf.plot().show()
pf.stats()

Start                               2023-01-01 00:10:00
End                                 2023-05-08 23:59:00
Period                                127 days 01:04:00
Start Value                                       100.0
End Value                                    154.499742
Total Return [%]                              54.499742
Benchmark Return [%]                          19.252683
Max Gross Exposure [%]                            100.0
Total Fees Paid                               15.643441
Max Drawdown [%]                              17.224557
Max Drawdown Duration                  33 days 19:42:00
Total Trades                                        196
Total Closed Trades                                 196
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  69.387755
Best Trade [%]                                 3.309081
Worst Trade [%]                               -8

Many other conditions can be added, such as stop losses, take profits, trailing stops, different entry/exit criteria, etc., and tested quickly on large historical datasets.